In [1]:
import pandas as pd
import json
from tqdm import trange

In [2]:
def read_jsonl(file_path):
    result_list = []
    with open(file_path, 'r') as f:
        f.seek(0, 2)
        eof = f.tell()
        f.seek(0, 0)
        while f.tell() < eof:
            result_list.append(json.loads(f.readline()))
    return result_list

In [3]:
df_label = pd.read_csv("/run/media/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-negbio.csv")
df_dicom = pd.read_csv("/run/media/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-metadata.csv")
train_metadata_list = read_jsonl("/run/media/mimic-pa-512/train/metadata.jsonl")
valid_metadata_list = read_jsonl("/run/media/mimic-pa-512/valid/metadata.jsonl")
df_train_meta = pd.DataFrame(train_metadata_list)
df_valid_meta = pd.DataFrame(valid_metadata_list)

In [4]:
df_label = df_label.fillna(0)
df_label = df_label.astype('int32')
df_label.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,10000032,53189527,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,10000032,53911762,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,10000032,56699142,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,10000764,57375967,0,0,1,0,0,0,0,0,0,0,0,-1,0,0


In [5]:
df_dicom.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


In [6]:
df_valid_meta.head()

,file_name,impression
0,0fe0f450-979ab2eb-ec87fcd3-c1660ba9-18517d0a.jpg,Right mid lung rounded opacity may represent a...
1,e75dad5d-03a961c6-602a953c-9bb24d1f-aaa47f83.jpg,"No radiographic evidence of pneumonia, intrath..."
2,02f3df2f-ff2bc640-5f173dca-eaff305d-73b20ae1.jpg,No acute cardiopulmonary abnormality.
3,4a8eb791-9bebcee1-8b01dcce-835b9a48-5d8896da.jpg,No acute cardiopulmonary process.
4,737caa62-adda5c8a-ad024448-5d85cbab-ec10a535.jpg,No acute cardiopulmonary process.


In [7]:
df_valid_meta.describe()

,file_name,impression
count,7868,7868
unique,7868,4020
top,0fe0f450-979ab2eb-ec87fcd3-c1660ba9-18517d0a.jpg,No acute cardiopulmonary process.
freq,1,1444


In [13]:
dicom_id = "02f3df2f-ff2bc640-5f173dca-eaff305d-73b20ae1"


dicom_file_name = f"{dicom_id}.jpg"
impression = df_valid_meta[df_valid_meta['file_name']==dicom_file_name]['impression'].to_list()[0]
print(impression)

_dicom_index = df_dicom[df_dicom['dicom_id']==dicom_id].index
_dicom_row = df_dicom.loc[_dicom_index]
subject_id = int(_dicom_row['subject_id'].iloc[0])
study_id = int(_dicom_row['study_id'].iloc[0])
_label_index = df_label[(df_label['study_id']==study_id) & (df_label['subject_id']==subject_id)].index
row = df_label.loc[_label_index]
row


Bilateral lower lobe bronchopneumonia, unchanged in appearance since prior examination.  RECOMMENDATION(S):  Recommend short interval followup chest radiograph ___ weeks after completion of treatment to assess for resolution.


,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
205003,19000174,52712438,0,0,0,0,0,0,0,0,0,0,0,1,0,0
